## Étape 1 : Montage de Google Drive et Extraction du Dataset

### 1.1 Montage de Google Drive
La première étape consiste à monter Notre Google Drive dans Google Colab pour pouvoir accéder à vos fichiers stockés. Ce code utilise le module `google.colab.drive` pour monter votre Drive dans l'environnement Colab.




In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip "/content/drive/MyDrive/archive (8).zip" -d "/content/dataset"

## Étape 2 : Chargement des Datasets d'Entraînement et de Validation

### 2.1 Définition des Paramètres
Avant de charger les datasets, nous définissons les paramètres nécessaires tels que les chemins vers les répertoires des données d'entraînement et de validation, la taille des images, ainsi que la taille des lots (batch size) pour l'entraînement.


In [ ]:
import tensorflow as tf
training_path = "/content/dataset/real_vs_fake/real-vs-fake/train"
validation_path = "/content/dataset/real_vs_fake/real-vs-fake/valid"
img_size = (224, 224)
batch_size = 32

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    training_path,
    labels="inferred",
    label_mode="int",
    image_size=img_size,
    shuffle=True,
    batch_size=batch_size
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    validation_path,
    labels="inferred",
    label_mode="int",
    image_size=img_size,
    shuffle=True,
    batch_size=batch_size
)




In [ ]:
"sks"

## Étape 3 : Prétraitement et Augmentation des Images

### 3.1 Définition de la Fonction de Prétraitement
Pour améliorer la généralisation du modèle, nous appliquons des transformations d'augmentation d'images. Ces transformations permettent d'introduire de la variabilité dans les données d'entraînement et d'éviter le sur-apprentissage (overfitting). Les transformations appliquées sont :

- **Flip horizontal aléatoire** : Permet de retourner l'image de manière aléatoire sur l'axe horizontal.
- **Contraste aléatoire** : Modifie le contraste de l'image dans une plage définie.
- **Luminosité aléatoire** : Modifie la luminosité de l'image de manière aléatoire.
- **Normalisation** : Convertit l'image en valeurs entre 0 et 1 en divisant par 255.


In [ ]:

def preprocess_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = tf.image.random_brightness(image, 0.1)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label
train_dataset = train_dataset.map(preprocess_image)
validation_dataset = validation_dataset.map(preprocess_image)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(tf.data.AUTOTUNE)

## Étape 4 : Exploration des Données (EDA)

### 4.1 Visualisation d'un Lot d'Images
Avant de commencer l'entraînement du modèle, il est important de visualiser les données pour vérifier leur qualité et leur distribution. Ici, nous récupérons un lot d'images et de labels à partir du dataset d'entraînement et les affichons à l'aide de `matplotlib`.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

image_batch, label_batch = next(iter(train_dataset))


def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10, 10))
    for n in range(min(25, len(image_batch))):
        ax = plt.subplot(5, 5, n + 1)
        plt.imshow(image_batch[n])
        plt.title(int(label_batch[n]))
        plt.axis('off')
    plt.show()

show_batch(image_batch.numpy(), label_batch.numpy())

class_names = train_dataset.class_names
train_labels = np.concatenate([y for x, y in train_dataset], axis=0)
validation_labels = np.concatenate([y for x, y in validation_dataset], axis=0)


def plot_class_distribution(labels, dataset_type):
  unique, counts = np.unique(labels, return_counts=True)
  plt.figure(figsize=(8, 6))
  plt.bar(class_names, counts)
  plt.title(f'Class Distribution for {dataset_type} Dataset')
  plt.xlabel('Class Labels')
  plt.ylabel('Number of Images')
  plt.show()
plot_class_distribution(train_labels, 'Training')
plot_class_distribution(validation_labels, 'Validation')

for images, labels in train_dataset.take(1):
  print("Image shape:", images.shape)



## Étape 5 : Création du Modèle avec Fine-Tuning

### 5.1 Importation du Modèle de Base (ResNet50)
Dans cette étape, nous utilisons un modèle pré-entraîné de ResNet50 comme base. Ce modèle a été formé sur ImageNet et est utilisé pour extraire des caractéristiques d'images. Nous chargeons le modèle sans sa couche finale (`include_top=False`) pour pouvoir y ajouter nos propres couches de classification.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

base_model = tf.keras.applications.ResNet50(
    weights="imagenet",
    input_shape=(224, 224, 3),
    include_top=False
)
base_model.trainable = True
fine_tune_at = 20
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")  # Binary classification
])


model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=0.0001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,770,433 (94.49 MB)

 Trainable params: 24,614,593 (93.90 MB)

 Non-trainable params: 155,840 (608.75 KB)

## Étape 6 : Entraînement du Modèle

### 6.1 Initialisation de l'EarlyStopping
Dans cette étape, nous utilisons la technique **EarlyStopping** pour arrêter l'entraînement du modèle lorsque la performance sur l'ensemble de validation cesse de s'améliorer. Cela permet d'éviter le sur-apprentissage et de sauvegarder le meilleur modèle.

In [ ]:

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10,
    callbacks=[early_stopping]
)



## Étape 7 : Préparation du Jeu de Données de Test

### 7.1 Chargement du Jeu de Données de Test
Dans cette étape, nous chargeons le jeu de données de test à partir du répertoire spécifié. Le jeu de données de test est utilisé pour évaluer la performance finale du modèle après l'entraînement.


In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/real_vs_fake/real-vs-fake/test',
    labels="inferred",
    label_mode="int",
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

test_dataset = test_dataset.map(preprocess_image)

test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)


## Étape 8 : Évaluation du Modèle sur le Jeu de Données de Test

### 8.1 Évaluation du Modèle
Une fois que le modèle a été entraîné, nous procédons à son évaluation sur le jeu de données de test. Cela nous permet de mesurer la performance finale du modèle en termes de perte (loss) et de précision (accuracy).


In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


## Étape 9 : Sauvegarde du Modèle Entraîné

### 9.1 Sauvegarder le Modèle
Une fois que le modèle a été entraîné et évalué, il est essentiel de le sauvegarder pour pouvoir l'utiliser ultérieurement sans avoir à le réentraîner. La méthode **model.save()** permet de sauvegarder le modèle dans un fichier.

In [ ]:
model.save('mon_modele.keras')
